**Proceso de ETL: Steam_games**

**Cargamos las librerias**

In [19]:
import pandas as pd
import numpy as np
import json
import html
import gzip
import warnings
warnings.filterwarnings("ignore")

**Extraction y primera exploración:**

Datos de archivo JSON se convierten a Dataframe para examinar su contenido.

In [21]:
games = []
with gzip.open('../Archivos Json/steam_games.json.gz', 'rt', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        games.append(data)

# Convertirlo a un dataframe
df_games = pd.DataFrame(games)

In [23]:
df_games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [26]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [28]:
#Observamos que todos nuestros tipos de datos son objetos
df_games["tags"][120000]

['Action', 'RPG', 'Indie', 'Shooter', 'Isometric']

In [29]:
df_games["early_access"].unique()

array([nan, False, True], dtype=object)

In [30]:
df_games["price"].unique()

array([nan, 4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, 'Free', 10.99, 1.59, 14.99, 1.99, 59.99, 8.99, 6.99,
       7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25, 24.99,
       17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo', 'Play for Free!',
       34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99, 79.99,
       49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99, 149.99, 771.71,
       'Install Now', 21.99, 89.99, 'Play WARMACHINE: Tactics Demo', 0.98,
       139.92, 4.29, 64.99, 'Free Mod', 54.99, 74.99, 'Install Theme',
       0.89, 'Third-party', 0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0,
       5.49, 23.99, 49.0, 20.99, 10.93, 1.39, 'Free HITMAN™ Holiday Pack',
       36.99, 4.49, 2.0, 4.0, 9.0, 234.99, 1.95, 1.5, 199.0, 189.0, 6.66,
       27.99, 10.49, 129.99, 179.0, 26.99, 399.99, 31.99, 399.0, 20.0,
       40.0, 3.33, 199.99, 22.99, 320.0, 38.85, 71.7, 59.95, 995.0, 27.49,
       3.39, 6.0, 19.95, 499.99, 16.06, 4.68, 131.4, 44.

In [31]:
df_games["specs"][120000]

['Single-player']

In [32]:
df_games["genres"][120000]

['Action', 'Indie', 'RPG']

**Verficamos datos nulos**

In [33]:
df_games.isnull().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [34]:
#Eliminamos las filas que solo contienen valores nulos es decir que no aportan nada
df_games = df_games.dropna(how="all").reset_index(drop=True)

In [35]:
df_games.shape

(32135, 13)

In [36]:
#Volvemos a verificar que variables tienen valores nulos y cuanto representan
cantidad_nulos= df_games.isnull().sum()
print(cantidad_nulos)

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64


In [37]:
#Aqui eliminamos los valores nulos que se encuentran en la variable id
df_games.dropna(subset=['id'], inplace=True)

In [38]:
cantidad_filas = len(df_games)
porcentaje_nulos_columnas= (cantidad_nulos/cantidad_filas)*100
porcentaje_nonulos_columnas = 100 - porcentaje_nulos_columnas
resumen_nulos = pd.DataFrame({"Porcentaje de valores nulos":porcentaje_nulos_columnas, 
                              "Porcentaje de valores no nulos":porcentaje_nonulos_columnas})
resumen_nulos

,Porcentaje de valores nulos,Porcentaje de valores no nulos
publisher,25.058351,74.941649
genres,10.216911,89.783089
app_name,0.006224,99.993776
title,6.379734,93.620266
url,0.000000,100.000000
release_date,6.432639,93.567361
tags,0.507267,99.492733
reviews_url,0.006224,99.993776
specs,2.085084,97.914916
price,4.285314,95.714686


**Filas duplicadas**

En este caso la columna en la que se quiere verificar si tiene duplicados es la columna . "id" dado que es una variable identificatoria de usuario

In [39]:
df_games[df_games["id"].duplicated()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games


In [40]:
#Eliminamos los datos duplicados pertenecientes a id
df_games = df_games.drop_duplicates(subset='id', keep='first')

In [41]:
df_games.isnull().sum()

publisher       8051
genres          3282
app_name           1
title           2049
url                0
release_date    2066
tags             162
reviews_url        0
specs            669
price           1377
early_access       0
id                 0
developer       3298
dtype: int64

**Variable Price**

En este caso la columna como evidenciamos antes tiene dos tipos de datos que son str y float, para poder trabajar con ella necesitamos que los valores queden como float analizando nos damos cuenta que los valores que estan en str indican que el juego fue gratis por lo que le vamos a asignar un precio de cero

In [42]:
df_games["price_number"] = pd.to_numeric(df_games["price"], errors="coerce")
df_games["price_number"] = pd.to_numeric(df_games["price_number"], errors="coerce")
df_games["price_number"] = df_games["price_number"].fillna(0)

In [43]:
df_games["price_number"].dtype

dtype('float64')

**Variable Release date**

En este caso lo que a nosotros nos va a importar es el año de lanzamiento por lo que vamos a crear una nueva columna para este fin

In [44]:
df_games["release_date"].isnull().sum()

2066

In [45]:
df_games["release_date"].unique()

array(['2018-01-04', '2017-07-24', '2017-12-07', ..., '2016-11-19',
       'January 2018', '2018-10-01'], dtype=object)

In [46]:
invalid_dates = df_games["release_date"][~df_games["release_date"].str.match(r"\d{4}-\d{2}-\d{2}", na=False, case=False, flags=0)]
df_invalid_dates = pd.DataFrame({"invalid_dates": invalid_dates})
df_invalid_dates

,invalid_dates
4,NaN
10,Soon..
11,NaN
19,NaN
20,NaN
...,...
32087,Coming Soon
32088,NaN
32097,2016
32123,January 2018


En este caso cambiamos el formato de la columna release_date para luego crear la columna release_year que es la variable que nos importa para el proximo analisis

In [47]:
df_games["release_date"] = pd.to_datetime(df_games["release_date"], errors="coerce", format='%Y-%m-%d', exact=False)
mean_release_date = df_games["release_date"].mean()
df_games["release_date_filled"] = df_games["release_date"].fillna(mean_release_date)
df_games["release_year"] = df_games["release_date_filled"].dt.year
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,price_number,release_date_filled,release_year
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,4.99,2018-01-04 00:00:00.000000000,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,0.00,2018-01-04 00:00:00.000000000,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,0.00,2017-07-24 00:00:00.000000000,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,0.99,2017-12-07 00:00:00.000000000,2017
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaT,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,2.99,2015-04-22 12:55:41.796600064,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",1.99,2018-01-04 00:00:00.000000000,2018
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,4.99,2018-01-04 00:00:00.000000000,2018
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,1.99,2018-01-04 00:00:00.000000000,2018
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns",4.99,2017-09-02 00:00:00.000000000,2017


In [48]:
df_games["release_year"].dtype

dtype('int64')

In [49]:
df_games["release_year"].unique()

array([2018, 2017, 2015, 1997, 1998, 2016, 2006, 2005, 2003, 2007, 2002,
       2000, 1995, 1996, 1994, 2001, 1993, 2004, 1999, 2008, 2009, 1992,
       1989, 2010, 2011, 2013, 2012, 2014, 1983, 1984, 1990, 1988, 1991,
       1985, 1982, 1987, 1981, 1986, 2021, 2019, 1975, 1970, 1980],
      dtype=int64)

In [50]:
df_games["release_year"].isnull().sum()

0

**Eliminamos las columnas que no seran necesarias para nuestro analisis y posterior modelado para optimizar el dataframe**

In [51]:
df_games.drop(["release_date", "price","url","publisher","reviews_url","specs","early_access","release_date_filled"], axis=1, inplace=True)
df_games

,genres,app_name,title,tags,id,developer,price_number,release_year
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",761140,Kotoshiro,4.99,2018
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,"[Free to Play, Strategy, Indie, RPG, Card Game...",643980,Secret Level SRL,0.00,2018
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...",670290,Poolians.com,0.00,2017
3,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,"[Action, Adventure, Casual]",767400,彼岸领域,0.99,2017
4,NaN,Log Challenge,NaN,"[Action, Indie, Casual, Sports]",773570,NaN,2.99,2015
...,...,...,...,...,...,...,...,...
32130,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,"[Strategy, Indie, Casual, Simulation]",773640,"Nikita ""Ghost_RUS""",1.99,2018
32131,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,"[Strategy, Indie, Casual]",733530,Sacada,4.99,2018
32132,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,"[Indie, Simulation, Racing]",610660,Laush Dmitriy Sergeevich,1.99,2018
32133,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",658870,"xropi,stev3ns",4.99,2017


**Variable Genres**

In [52]:
df_games["genres"].isnull().sum()

3282

Vamos a imputar los valores nulos de la variable genres con los valores pertenecientes a la variable tags pero los cuales si pertenecen a un genero

In [53]:
generos = []
for genres_list in df_games["genres"].dropna():
    for genre in genres_list:
        if genre not in generos:
            generos.append(genre)
generos

['Action',
 'Casual',
 'Indie',
 'Simulation',
 'Strategy',
 'Free to Play',
 'RPG',
 'Sports',
 'Adventure',
 'Racing',
 'Early Access',
 'Massively Multiplayer',
 'Animation &amp; Modeling',
 'Video Production',
 'Utilities',
 'Web Publishing',
 'Education',
 'Software Training',
 'Design &amp; Illustration',
 'Audio Production',
 'Photo Editing',
 'Accounting']

In [54]:
df_games["tags"]=df_games["tags"].dropna().apply(lambda lista: [g for g in lista if g in generos])

In [55]:
df_games["genres"].fillna(df_games["tags"], inplace=True)

In [56]:
df_games["genres"].isnull().sum()

138

Aunque imputamos los registros podemos verificar que siguen quedando registros nulos que se trataran mas adelante

Procedemos a eliminar la columna tags dado que ya no es necesaria

In [57]:
df_games = df_games.drop("tags", axis=1)
df_games

,genres,app_name,title,id,developer,price_number,release_year
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,761140,Kotoshiro,4.99,2018
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,643980,Secret Level SRL,0.00,2018
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,670290,Poolians.com,0.00,2017
3,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,767400,彼岸领域,0.99,2017
4,"[Action, Indie, Casual, Sports]",Log Challenge,NaN,773570,NaN,2.99,2015
...,...,...,...,...,...,...,...
32130,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,773640,"Nikita ""Ghost_RUS""",1.99,2018
32131,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,733530,Sacada,4.99,2018
32132,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,610660,Laush Dmitriy Sergeevich,1.99,2018
32133,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,658870,"xropi,stev3ns",4.99,2017


In [58]:
df_games["genres"].head()

0        [Action, Casual, Indie, Simulation, Strategy]
1                 [Free to Play, Indie, RPG, Strategy]
2    [Casual, Free to Play, Indie, Simulation, Sports]
3                          [Action, Adventure, Casual]
4                      [Action, Indie, Casual, Sports]
Name: genres, dtype: object

Como podemos observar la variable contiene una lista de generos en cada registro por lo que nos encontramos con datos anidados que es necesario desanidarlos, en este caso para realizar el cometido se utilizara explode para desanidar

In [60]:
df_games["main_genre"] = df_games["genres"].apply(lambda x: x[0] if isinstance(x, list) and x else None)
df_unique_genre = df_games.drop_duplicates(["app_name", "title", "id", "developer", "price_number", "release_year"])
df_unique_genre = df_unique_genre.drop("genres", axis=1)
df_unique_genre

,app_name,title,id,developer,price_number,release_year,main_genre
0,Lost Summoner Kitty,Lost Summoner Kitty,761140,Kotoshiro,4.99,2018,Action
1,Ironbound,Ironbound,643980,Secret Level SRL,0.00,2018,Free to Play
2,Real Pool 3D - Poolians,Real Pool 3D - Poolians,670290,Poolians.com,0.00,2017,Casual
3,弹炸人2222,弹炸人2222,767400,彼岸领域,0.99,2017,Action
4,Log Challenge,NaN,773570,NaN,2.99,2015,Action
...,...,...,...,...,...,...,...
32130,Colony On Mars,Colony On Mars,773640,"Nikita ""Ghost_RUS""",1.99,2018,Casual
32131,LOGistICAL: South Africa,LOGistICAL: South Africa,733530,Sacada,4.99,2018,Casual
32132,Russian Roads,Russian Roads,610660,Laush Dmitriy Sergeevich,1.99,2018,Indie
32133,EXIT 2 - Directions,EXIT 2 - Directions,658870,"xropi,stev3ns",4.99,2017,Casual


In [61]:
df_unique_genre["main_genre"].isnull().sum()

1030

In [62]:
df_unique_genre["main_genre"] = df_unique_genre["main_genre"].fillna("Sin Dato")
df_unique_genre["main_genre"].isnull().sum()

0

**Variables App_name, title y developer**

In [63]:
columnas_a_llenar = ["app_name", "title"]
df_unique_genre[columnas_a_llenar] = df_unique_genre[columnas_a_llenar].fillna("Sin datos")

In [64]:
son_iguales = (df_unique_genre["app_name"] == df_unique_genre["title"]).all()

if son_iguales:
    print("Las columnas 'app_name' y 'title' son iguales en todos los registros.")
else:
    print("Las columnas 'app_name' y 'title' no son iguales en todos los registros.")

Las columnas 'app_name' y 'title' no son iguales en todos los registros.


In [65]:
df_unique_genre["son_iguales"] = df_unique_genre["app_name"] == df_unique_genre["title"]
filas_no_iguales = df_unique_genre[df_unique_genre["son_iguales"] == False]
filas_no_iguales

,app_name,title,id,developer,price_number,release_year,main_genre,son_iguales
4,Log Challenge,Sin datos,773570,NaN,2.99,2015,Action,False
11,Icarus Six Sixty Six,Sin datos,724910,NaN,0.00,2015,Casual,False
19,After Life VR,Sin datos,772590,NaN,4.99,2015,Early Access,False
20,Kitty Hawk,Sin datos,640250,NaN,2.99,2015,Early Access,False
22,Mortars VR,Sin datos,711440,NaN,0.99,2015,Early Access,False
...,...,...,...,...,...,...,...,...
32073,Tank of War-VR,Sin datos,745900,NaN,19.99,2015,Action,False
32076,Flappy Arms,Sin datos,764110,NaN,1.99,2015,Casual,False
32077,SpaceWalker,Sin datos,705860,NaN,0.00,2015,Early Access,False
32085,LIV Client,Sin datos,755540,NaN,0.00,2015,Video Production,False


En el caso de los Developers es una variable importante para el analisis posterior por lo que los valores nulos que no aportan informacion se proceden a eliminar

In [66]:
df_unique_genre.dropna(subset=["developer"], inplace= True)

In [67]:
df_unique_genre.isnull().sum()

app_name        0
title           0
id              0
developer       0
price_number    0
release_year    0
main_genre      0
son_iguales     0
dtype: int64

Como pudimos evidenciar las columnas app_name y title solo difieren que en la columna title los registros son diferentes es porque tienen las letras amp por lo que solo nos vamos a quedar con la columna app_name

In [68]:
df_unique_genre.drop(columns=["title"],axis=1,inplace=True)

In [69]:
df_unique_genre

,app_name,id,developer,price_number,release_year,main_genre,son_iguales
0,Lost Summoner Kitty,761140,Kotoshiro,4.99,2018,Action,True
1,Ironbound,643980,Secret Level SRL,0.00,2018,Free to Play,True
2,Real Pool 3D - Poolians,670290,Poolians.com,0.00,2017,Casual,True
3,弹炸人2222,767400,彼岸领域,0.99,2017,Action,True
5,Battle Royale Trainer,772540,Trickjump Games Ltd,3.99,2018,Action,True
...,...,...,...,...,...,...,...
32129,Kebab it Up!,745400,Bidoniera Games,1.99,2018,Action,True
32130,Colony On Mars,773640,"Nikita ""Ghost_RUS""",1.99,2018,Casual,True
32131,LOGistICAL: South Africa,733530,Sacada,4.99,2018,Casual,True
32132,Russian Roads,610660,Laush Dmitriy Sergeevich,1.99,2018,Indie,True


In [70]:
df_unique_genre.drop(["son_iguales"], axis=1, inplace=True)

In [71]:
df_unique_genre.head(2)

,app_name,id,developer,price_number,release_year,main_genre
0,Lost Summoner Kitty,761140,Kotoshiro,4.99,2018,Action
1,Ironbound,643980,Secret Level SRL,0.00,2018,Free to Play


In [72]:
df_unique_genre.isnull().sum()

app_name        0
id              0
developer       0
price_number    0
release_year    0
main_genre      0
dtype: int64

**Cargamos el dataset limpio**

In [73]:
archivo_limpio = 'steam_games_limpio.csv'
df_unique_genre.to_csv(archivo_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {archivo_limpio}')

Se guardó el archivo steam_games_limpio.csv
